In [1]:
import torch
import torch.nn as nn
import torch.utils.data
import h5py
import numpy as np
import scipy.io as sio

In [2]:
# Hyper-parameters
latent_size = 5184
hidden_size = 1728
image_size = 216
num_epochs = 200
batch_size = 128
sample_dir = '/home/nhjeong/MLPGAN/db'

In [3]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size))


In [5]:
# Device configuration
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:3" if use_cuda else "cpu")

In [6]:
G = torch.load('MLPGAN_aug_noGAN_300epochs_Adam_weight2.pkl')

RuntimeError: CUDA error: out of memory

In [5]:
class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, train=True):
        
        self.train = train
        
        if self.train:
            self.train_X_mat = h5py.File('/home/nhjeong/MLPGAN/db/db4.mat', 'r')
            self.train_X_input = self.train_X_mat['db'][:]


            self.train_Y_mat = h5py.File('/home/nhjeong/MLPGAN/db/gt4.mat', 'r')
            self.train_Y_input = self.train_Y_mat['gt'][:]

            self.train_X_mat.close()
            self.train_Y_mat.close()

        else:
            self.test_X_mat = h5py.File('/home/nhjeong/MLPGAN/db/test_db4.mat', 'r')
            self.test_X_input = self.test_X_mat['test_db'][:]
#             self.test_X_input = np.transpose(self.test_X_input)            

            self.test_Y_mat = h5py.File('/home/nhjeong/MLPGAN/db/test_gt4.mat', 'r')
            self.test_Y_input = self.test_Y_mat['test_gt'][:]
#             self.test_Y_input = np.transpose(self.test_Y_input)            

            self.test_X_mat.close()
            self.test_Y_mat.close()
        
        
        
    def __len__(self):
        if self.train:
            return self.train_X_input.shape[0]
        else:
            return self.test_X_input.shape[0]
    
    def __getitem__(self, index):
        if self.train:
            raw, target = self.train_X_input[index,], self.train_Y_input[index,]
        else:
            raw, target = self.test_X_input[index,], self.test_Y_input[index,]
            
        return raw, target
    
    
    
# trainset = MyDataset(train=True)
testset = MyDataset(train=False)

# trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
# testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=13824, shuffle=False) 

In [6]:

#cudnn.benchmark = True

output = G(torch.tensor(testset.test_X_input).to(device))

In [7]:
test_result = output.cpu().detach().numpy()

In [8]:
sio.savemat('/home/nhjeong/MLPGAN/Test_300epochs_Adam_result2.mat', {'test_result': test_result})

In [26]:
test_result

array([[-0.7409611 , -0.7828354 , -0.7856567 , ..., -0.7588253 ,
        -0.7933631 , -0.7382162 ],
       [-0.7687777 , -0.7987708 , -0.80388933, ..., -0.77747977,
        -0.81061625, -0.75554365],
       [-0.80422765, -0.83781683, -0.85696054, ..., -0.8168291 ,
        -0.85702556, -0.8026087 ],
       ...,
       [-0.74996   , -0.7969581 , -0.79548633, ..., -0.7697613 ,
        -0.8034985 , -0.7540104 ],
       [-0.7906202 , -0.8081559 , -0.81907123, ..., -0.79320455,
        -0.8249649 , -0.7707753 ],
       [-0.753826  , -0.79313433, -0.79628855, ..., -0.7727097 ,
        -0.8020096 , -0.7568225 ]], dtype=float32)